In [1]:
from IPython.core.debugger import set_trace

In [65]:
import pandas as pd
import numpy as np

Loading lists of samples

In [3]:
def set_class_metrics(sample):
    return sample[sample["level"] == "class"]

In [4]:
def set_project_methods(sample):
    return sample[sample["level"] == "project"]

In [5]:
def set_other_metrics(sample_metrics, projects_metrics):
    sample_metrics["JAVA"] = projects_metrics["numberJavaFiles"].values
    sample_metrics["CLASSES"] = projects_metrics["CountDeclClass"].values
    sample_metrics["PKG"] = projects_metrics["packages"].values
    sample_metrics["BWR"] = (sample_metrics["Buse Readability"].values * projects_metrics["numberJavaFiles"].values) / projects_metrics["CountDeclClass"].values
    sample_metrics["SAR"] = (sample_metrics["Scalabrino Readability"].values * projects_metrics["numberJavaFiles"].values) / projects_metrics["CountDeclClass"].values
    return sample_metrics

In [6]:
def set_method_metrics(sample, projects_metrics, sample_metrics):
    method_metrics = sample[sample["level"] == "method"]
    
    sample_metrics["FANIN"] = (method_metrics["CountInput"].values * sample_metrics["CountDeclMethodAll"].values) / projects_metrics["CountDeclClass"].values
    sample_metrics["FANOUT"] = (method_metrics["CountOutput"].values * sample_metrics["CountDeclMethodAll"].values) / projects_metrics["CountDeclClass"].values
    sample_metrics["CC"] = (method_metrics["Cyclomatic"].values * sample_metrics["CountDeclMethodAll"].values) / projects_metrics["CountDeclClass"].values
    sample_metrics["CCM"] = (method_metrics["CyclomaticModified"].values * sample_metrics["CountDeclMethodAll"].values) / projects_metrics["CountDeclClass"].values
    sample_metrics["CCS"] = (method_metrics["CyclomaticStrict"].values * sample_metrics["CountDeclMethodAll"].values) / projects_metrics["CountDeclClass"].values
    sample_metrics["EC"] = (method_metrics["Essential"].values * sample_metrics["CountDeclMethodAll"].values) / projects_metrics["CountDeclClass"].values
    sample_metrics["KNTS"] = (method_metrics["Knots"].values * sample_metrics["CountDeclMethodAll"].values) / projects_metrics["CountDeclClass"].values
    sample_metrics["NPATH"] = (method_metrics["CountPath"].values * sample_metrics["CountDeclMethodAll"].values) / projects_metrics["CountDeclClass"].values
    sample_metrics["NPLOG"] = (method_metrics["CountPathLog"].values * sample_metrics["CountDeclMethodAll"].values) / projects_metrics["CountDeclClass"].values    
    return sample_metrics

In [7]:
def get_commit_date(sample_metrics):
    return pd.to_datetime(sample_metrics['commitDate'].astype(str).str[:-6])

In [8]:
def getSampleDataframe(name):
    sample = pd.read_csv("../2-ExtractingMetrics/metrics/"+name+".csv", parse_dates=True)
  
    #class metrics
    sample_metrics = set_class_metrics(sample)
    
    #project metrics
    projects_metrics = set_project_methods(sample)
    
    #other metrics
    sample_metrics = set_other_metrics(sample_metrics, projects_metrics)
    
    #method metrics
    sample_metrics = set_method_metrics(sample, projects_metrics, sample_metrics)
    
    sample_metrics['commitDate'] = get_commit_date(sample_metrics)
    
    sample_metrics.set_index("commitDate", inplace=True)

#     sample_metrics.dropna(axis=1, how="all", inplace=True)
    
    sample_metrics = sample_metrics.groupby([(sample_metrics.index.year), (sample_metrics.index.month)]).mean()
    
    first_date = sample_metrics.index.to_flat_index()[0]
    last_date = sample_metrics.index.to_flat_index()[-1]
        
    generic = pd.DataFrame(index=pd.date_range(start=f"{first_date[0]}-{first_date[1]}-01", end=f"{last_date[0]}-{last_date[1]}-01", freq="M", name="commitDate"))
    
    generic = generic.groupby([(generic.index.year), (generic.index.month)]).sum()
    
    return pd.concat([generic, sample_metrics], axis=1).fillna(method='ffill').dropna(how="all")

For each framework, load metrics per samples and group by month

In [9]:
def loadMetrics(samplesList):
    samples = dict()
    for sample in samplesList:
        samples[sample] = getSampleDataframe(sample)
           
        samples[sample]["PL"] = samples[sample]["CountLine"]
        del samples[sample]["CountLine"]
        
        samples[sample]["LOC"] = samples[sample]["CountLineCode"]
        del samples[sample]["CountLineCode"]
        
        samples[sample]["DLOC"] = samples[sample]["CountLineCodeDecl"]
        del samples[sample]["CountLineCodeDecl"]
        
        samples[sample]["ELOC"] = samples[sample]["CountLineCodeExe"]
        del samples[sample]["CountLineCodeExe"]
        
        samples[sample]["CLOC"] = samples[sample]["CountLineComment"]
        del samples[sample]["CountLineComment"]
        
        samples[sample]["BL"] = samples[sample]["CountLineBlank"]
        del samples[sample]["CountLineBlank"]
        
        samples[sample]["NV"] = samples[sample]["CountDeclClassVariable"]
        del samples[sample]["CountDeclClassVariable"]
        
        samples[sample]["AM"] = samples[sample]["CountDeclMethodAll"]
        
        samples[sample]["NPRM"] = samples[sample]["CountDeclMethodPublic"]
        del samples[sample]["CountDeclMethodPublic"]
        
        samples[sample]["NPM"] = samples[sample]["CountDeclMethodPrivate"]
        del samples[sample]["CountDeclMethodPrivate"]
        
        samples[sample]["PM"] = samples[sample]["CountDeclMethodProtected"]
        del samples[sample]["CountDeclMethodProtected"]
        
        samples[sample]["LM"] = samples[sample]["CountDeclMethod"]
        del samples[sample]["CountDeclMethod"]
        
        samples[sample]["STMT"] = samples[sample]["CountStmt"]
        del samples[sample]["CountStmt"]
        
        samples[sample]["DSTMT"] = samples[sample]["CountStmtDecl"]
        del samples[sample]["CountStmtDecl"]
        
        samples[sample]["ESTMT"] = samples[sample]["CountStmtExe"]
        del samples[sample]["CountStmtExe"]
        
        samples[sample]["CCR"] = samples[sample]["RatioCommentToCode"]
        del samples[sample]["RatioCommentToCode"]
        
        samples[sample]["WMC"] = samples[sample]["SumCyclomatic"]
        del samples[sample]["SumCyclomatic"]
        
        samples[sample]["NOC"] = samples[sample]["CountClassDerived"]
        del samples[sample]["CountClassDerived"]
        
        samples[sample]["DIT"] = samples[sample]["MaxInheritanceTree"]
        del samples[sample]["MaxInheritanceTree"]
        
        samples[sample]["LCOM"] = samples[sample]["PercentLackOfCohesion"]
        del samples[sample]["PercentLackOfCohesion"]
        
        samples[sample]["CBO"] = samples[sample]["CountClassCoupled"]
        del samples[sample]["CountClassCoupled"]
        
        samples[sample]["RFC"] = samples[sample]["CountDeclMethodAll"]
        del samples[sample]["CountDeclMethodAll"]
        
        samples[sample]["NIV"] = samples[sample]["CountDeclInstanceVariable"]
        del samples[sample]["CountDeclInstanceVariable"]
        
        del samples[sample]["AvgCyclomatic"]
        del samples[sample]["AvgCyclomaticModified"]
        del samples[sample]["AvgCyclomaticStrict"]
        del samples[sample]["AvgEssential"]
        del samples[sample]["AvgLine"]
        del samples[sample]["AvgLineBlank"]
        del samples[sample]["AvgLineCode"]
        del samples[sample]["AvgLineComment"]
        del samples[sample]["MaxCyclomatic"]
        del samples[sample]["MaxCyclomaticModified"]
        del samples[sample]["MaxCyclomaticStrict"]
        del samples[sample]["MaxEssential"]
        del samples[sample]["MaxNesting"]
        del samples[sample]["PercentLackOfCohesionModified"]
        del samples[sample]["SumCyclomaticModified"]
        del samples[sample]["SumCyclomaticStrict"]
        del samples[sample]["SumEssential"]
        del samples[sample]["CountClassBase"]
        del samples[sample]["CountClassCoupledModified"]
        del samples[sample]["CountDeclClassMethod"]
        del samples[sample]["CountDeclInstanceMethod"]
        del samples[sample]["CountDeclMethodDefault"]
        del samples[sample]["CountSemicolon"]
        del samples[sample]["Buse Readability"]
        del samples[sample]["Scalabrino Readability"]
        del samples[sample]["CountDeclClass"]
        del samples[sample]["CountDeclExecutableUnit"]
        del samples[sample]["CountDeclFile"]
        del samples[sample]["CountDeclFunction"]
        del samples[sample]["CountInput"]
        del samples[sample]["CountOutput"]
        del samples[sample]["CountPath"]
        del samples[sample]["CountPathLog"]
        del samples[sample]["Cyclomatic"]
        del samples[sample]["CyclomaticModified"]
        del samples[sample]["CyclomaticStrict"]
        del samples[sample]["Essential"]
        del samples[sample]["Knots"]
        del samples[sample]["MaxEssentialKnots"]
        del samples[sample]["numberJavaFiles"]
        del samples[sample]["MinEssentialKnots"]
        del samples[sample]["packages"]
        

    return samples

In [10]:
samples = pd.read_csv("../1-GettingQuestions/sampleQuestions.csv")
samples["full_name"] = samples["framework"] + "/" + samples["path"]
samples = samples["full_name"].unique()

In [11]:
metrics = loadMetrics(samples)

<ipython-input-5-42c1ef20c226>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_metrics["JAVA"] = projects_metrics["numberJavaFiles"].values
<ipython-input-5-42c1ef20c226>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_metrics["CLASSES"] = projects_metrics["CountDeclClass"].values
<ipython-input-5-42c1ef20c226>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

<ipython-input-5-42c1ef20c226>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_metrics["JAVA"] = projects_metrics["numberJavaFiles"].values
<ipython-input-5-42c1ef20c226>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_metrics["CLASSES"] = projects_metrics["CountDeclClass"].values
<ipython-input-5-42c1ef20c226>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

<ipython-input-5-42c1ef20c226>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_metrics["JAVA"] = projects_metrics["numberJavaFiles"].values
<ipython-input-5-42c1ef20c226>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_metrics["CLASSES"] = projects_metrics["CountDeclClass"].values
<ipython-input-5-42c1ef20c226>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

<ipython-input-6-87fb1d4ec40f>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_metrics["CCM"] = (method_metrics["CyclomaticModified"].values * sample_metrics["CountDeclMethodAll"].values) / projects_metrics["CountDeclClass"].values
<ipython-input-6-87fb1d4ec40f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_metrics["CCS"] = (method_metrics["CyclomaticStrict"].values * sample_metrics["CountDeclMethodAll"].values) / projects_metrics["CountDeclClass"].values
<ipython-input-6-87fb1d4ec40f>:9: 

<ipython-input-5-42c1ef20c226>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_metrics["JAVA"] = projects_metrics["numberJavaFiles"].values
<ipython-input-5-42c1ef20c226>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_metrics["CLASSES"] = projects_metrics["CountDeclClass"].values
<ipython-input-5-42c1ef20c226>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

...

<ipython-input-5-42c1ef20c226>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_metrics["JAVA"] = projects_metrics["numberJavaFiles"].values
<ipython-input-5-42c1ef20c226>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_metrics["CLASSES"] = projects_metrics["CountDeclClass"].values
<ipython-input-5-42c1ef20c226>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

<ipython-input-5-42c1ef20c226>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_metrics["JAVA"] = projects_metrics["numberJavaFiles"].values
<ipython-input-5-42c1ef20c226>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_metrics["CLASSES"] = projects_metrics["CountDeclClass"].values
<ipython-input-5-42c1ef20c226>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

<ipython-input-5-42c1ef20c226>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_metrics["JAVA"] = projects_metrics["numberJavaFiles"].values
<ipython-input-5-42c1ef20c226>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_metrics["CLASSES"] = projects_metrics["CountDeclClass"].values
<ipython-input-5-42c1ef20c226>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

<ipython-input-5-42c1ef20c226>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_metrics["JAVA"] = projects_metrics["numberJavaFiles"].values
<ipython-input-5-42c1ef20c226>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_metrics["CLASSES"] = projects_metrics["CountDeclClass"].values
<ipython-input-5-42c1ef20c226>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

<ipython-input-5-42c1ef20c226>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_metrics["JAVA"] = projects_metrics["numberJavaFiles"].values
<ipython-input-5-42c1ef20c226>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_metrics["CLASSES"] = projects_metrics["CountDeclClass"].values
<ipython-input-5-42c1ef20c226>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the document

Load questions

In [12]:
questions = pd.read_csv("../1-GettingQuestions/sampleQuestions.csv")

In [13]:
def clearQuestions(questions):
    questions["questions"] = 1
    questions["creationDate"] = pd.to_datetime(questions["creationDate"])
    questions.sort_values(by="creationDate", inplace=True)
    questions.set_index(questions["creationDate"], inplace=True)
    del questions["ownerUserId"]
    del questions["postTypeId"]
    del questions["acceptedAnswerId"]
    del questions["id"]
    del questions["creationDate"]
    questions["full_name"] = questions["framework"] +"/"+ questions["path"]
    return questions

In [14]:
questions = clearQuestions(questions)

In [15]:
def joinMetricsWithQuestions(metrics, questions):
    joint_metrics_questions = dict()
    for sample in metrics.keys():
        question = questions[questions["full_name"] == sample]
        question = question.groupby([(question.index.year), (question.index.month)]).sum()
        question["questions"] = question.cumsum()
        smp = metrics[sample]
        joint_metrics_questions[sample] = pd.concat([smp, question], axis=1)
        joint_metrics_questions[sample].fillna(method="ffill", inplace=True)
        joint_metrics_questions[sample].fillna(0, inplace=True)
    return joint_metrics_questions

In [16]:
joint_metrics_questions = joinMetricsWithQuestions(metrics, questions)

### Average increase or decrease metrics

In [87]:
variance = dict()
for sample in list(joint_metrics_questions.keys()):
    first_commit = joint_metrics_questions[sample].iloc[0]
    last_commit = joint_metrics_questions[sample].iloc[-1]
    for metric in first_commit.index:
        if not metric in variance.keys():
            variance[metric] = last_commit[metric]/first_commit[metric]
        else:
            sample_variance = last_commit[metric]/first_commit[metric]
            if np.isnan(sample_variance) or not np.isfinite(sample_variance):
                if not (last_commit[metric] == 0 and first_commit[metric] == 0):
                    variance[metric] = variance[metric] + (last_commit[metric] if not last_commit[metric] == 0 else first_commit[metric])
            else:
                variance[metric] = variance[metric] + sample_variance

<ipython-input-87-15fc752b0bae>:7: RuntimeWarning: divide by zero encountered in double_scalars
  variance[metric] = last_commit[metric]/first_commit[metric]
<ipython-input-87-15fc752b0bae>:9: RuntimeWarning: invalid value encountered in double_scalars
  sample_variance = last_commit[metric]/first_commit[metric]
<ipython-input-87-15fc752b0bae>:9: RuntimeWarning: divide by zero encountered in double_scalars
  sample_variance = last_commit[metric]/first_commit[metric]


In [88]:
for key in list(variance.keys()):
    variance[key] = ((variance[key] / len(joint_metrics_questions)) * 100) - 100

In [91]:
variance

{'JAVA': 208.61159312100625,
 'CLASSES': 299.2712804570824,
 'PKG': 95.99096766840864,
 'BWR': 3832.956279707585,
 'SAR': -20.37360167625954,
 'FANIN': 3.277607743723408,
 'FANOUT': 5.649000807094382,
 'CC': -6.4053789716840015,
 'CCM': -6.526176542467439,
 'CCS': -5.752037022864755,
 'EC': -6.503843923593166,
 'KNTS': -12.488898211000858,
 'NPATH': 19079.152727480367,
 'NPLOG': 35.08337802496237,
 'PL': 153.62961408934092,
 'LOC': 124.36729509779147,
 'DLOC': 65.27407256181849,
 'ELOC': 131.2891872540179,
 'CLOC': 65.85491172209035,
 'BL': 46.36536571807906,
 'NV': 73.40505467973622,
 'AM': 45.97493646662332,
 'NPRM': 36.33654970645793,
 'NPM': -3.472139298998428,
 'PM': -34.3256440205533,
 'LM': 35.980322969692764,
 'STMT': 89.04730208381099,
 'DSTMT': 60.87523063491429,
 'ESTMT': 84.9497372099448,
 'CCR': -1.6611890910823774,
 'WMC': 49.22033575965244,
 'NOC': -76.31082182983661,
 'DIT': -0.023323609288112834,
 'LCOM': 167.98590979206506,
 'CBO': 38.80496868994413,
 'RFC': 45.974936

## Normalize metrics

In [92]:
def normalize(metrics):
    return (metrics-metrics.min())/(metrics.max()-metrics.min())

In [93]:
def normalizeMetrics(metrics, listSamples):
    newMetrics = dict()
    for sample in listSamples:
        newMetrics[sample] = normalize(metrics[sample])
        newMetrics[sample].fillna(0, inplace=True)
    return newMetrics

In [94]:
normalized = normalizeMetrics(joint_metrics_questions, samples)

In [95]:
for sample in normalized.keys():
    normalized[sample].to_csv(f"{sample}.csv")

Number of Commits

In [96]:
dataframe = pd.DataFrame()
for sample in samples:
    dataframe = dataframe.append(pd.read_csv("../2-ExtractingMetrics/metrics/"+sample+".csv", parse_dates=True))
len(dataframe.groupby(by="commitSha"))

15184